In [9]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [123]:
files=['amazon_cells_labelled','imdb_labelled','yelp_labelled']
dataset={} #structure {'amazon': [[bad comments],[good comments]], ...}
for file in files:
    dataset[file]=[[] for i in range(2)]
    f = open(file+'.txt')
    for line in f:
        comment, label=line.split('\t')
        dataset[file][int(label)].append(comment)
    
for key, value in dataset.items():
    print('In',key,':',len(value[0]),'bad comments',len(value[1]),'good comments')

In amazon_cells_labelled : 500 bad comments 500 good comments
In imdb_labelled : 500 bad comments 500 good comments
In yelp_labelled : 500 bad comments 500 good comments


In [124]:
stopwords=nltk.corpus.stopwords.words('english')
stemmer=nltk.SnowballStemmer("english")
import re
for key, value in dataset.items():
    for comments in value:
        for i in range(len(comments)):
            lowered=comments[i].lower()
            word_list=re.findall(r"[a-z]+", lowered)
            filtered_words = [stemmer.stem(word) for word in word_list if word not in stopwords]
            comments[i] = filtered_words

In [125]:
trainData, trainLabel, testData, testLabel=[], [], [], []
for key, value in dataset.items():
    for i in range(2):
        trainData+=value[i][:400]
        trainLabel+=[i]*400
        testData+=value[i][400:]
        testLabel+=[i]*100

In [200]:
words=set()
for comment in trainData: words.update(set(comment))
words=list(words)
BoWTrainData=[]
for comment in trainData: 
    bag =[0 for _ in range(len(words))]
    for word in comment: 
        if word in words: bag[words.index(word)]+=1
    BoWTrainData.append(bag)
BoWTestData=[]
for comment in testData: 
    bag =[0 for _ in range(len(words))]
    for word in comment: 
        if word in words: bag[words.index(word)]+=1
    BoWTestData.append(bag)
# for i in range(2): print(bagsOfWords[i])

In [201]:
import numpy as np
logNormedBoWTrainData=np.log(np.array(BoWTrainData)+1)
logNormedBoWTestData=np.log(np.array(BoWTestData)+1)

In [202]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix,accuracy_score

logistic = LogisticRegression().fit(logNormedBoWTrainData, trainLabel)
gnb = GaussianNB().fit(logNormedBoWTrainData, trainLabel)
logistic_pred = logistic.predict(logNormedBoWTestData)
gnb_pred = gnb.predict(logNormedBoWTestData)
print('Using Logistic Regression:')
print(accuracy_score(testLabel, logistic_pred))
print(confusion_matrix(testLabel,logistic_pred))
print('Using Gaussian Naive Bayes:')
print(accuracy_score(testLabel, gnb_pred))
print(confusion_matrix(testLabel,gnb_pred))

(2400, 3373)
Using Logistic Regression:
0.821666666667
[[265  35]
 [ 72 228]]
Using Gaussian Naive Bayes:
0.638333333333
[[267  33]
 [184 116]]
